<a href="https://colab.research.google.com/github/klordo/nlp_homeworks/blob/hw2/nlp_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Как я понял, датасет полностью из ноутбука не загрузить, а только вручную. Не стал его прикреплять, т.к. он у всех один, но могу прикрепить, например ссылкой на диск (если так можно и нужно)

In [1]:
import pandas as pd
import spacy

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

In [ ]:
!python3 -m spacy download en_core_web_sm

In [3]:
data = pd.read_csv('spam_or_not_spam.csv', encoding='iso-8859-1').rename(columns={'email': 'text'})
data.sample(10)

,text,label
679,im feeling a bit farklempt having spent the n...,0
2866,take control of your computer with this top o...,1
2234,url URL date not supplied the launch is postpo...,0
2151,url URL date NUMBER NUMBER NUMBERtNUMBER NUMBE...,0
2748,dear homeowner interest rates are at their low...,1
1790,forwarded by nev dull nev sleepycat com forwa...,0
117,understand not enough caffeine absorbed yet th...,0
1490,hi i upgraded to NUMBER NUMBER now NUMBER NUMB...,0
773,on tue NUMBER sep NUMBER rohit khare wrote i s...,0
1816,use perl daily newsletter in this issue instal...,0


In [4]:
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words

In [5]:
%%time

data['cleaned_text'] = data['text'].apply(
    lambda x: ' '.join(
        token.lemma_.lower() for token in nlp(str(x)) if
        not token.is_stop
        and not token.is_punct
        and not token.is_digit
        and not token.like_email
        and not token.like_num
        and not token.is_space
    )
)
data.sample(10)

CPU times: user 2min 32s, sys: 1.21 s, total: 2min 33s
Wall time: 2min 49s


,text,label,cleaned_text
2032,url URL date NUMBER NUMBER NUMBERtNUMBER NUMBE...,0,url url date number number numbertnumber numbe...
1164,thanks paul that is the way i am doing right n...,0,thank paul way right like use mouse thing clue...
2638,hyperlink hyperlink hyperlink hyperlink,1,hyperlink hyperlink hyperlink hyperlink
222,hi all i ve decided at last to test the alsa s...,0,hi ve decide test alsa sound driver usual resu...
546,i m not sure which way to make the old bits c...,0,m sure way old bit url post b url archive b di...
608,at NUMBER NUMBER pm NUMBER on NUMBER NUMBER NU...,0,number number pm number number number number j...
1598,i m getting no servers available about half th...,0,m get server available half time day razor num...
234,not fortean but a moment in time all the same ...,0,fortean moment time url betamax video recorder...
2595,vc computers ltd ireland hosting from an irish...,1,vc computers ltd ireland host irish company li...
2680,NUMBER of women desire a bigger man from the ...,1,number woman desire big man creator longitude ...


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['label'], random_state=1234)

In [7]:
# Подобрал подходящие гиперпараметры
count_vectorizer = CountVectorizer(max_df=0.7, min_df=0.01)
tfidf_vectorizer = TfidfVectorizer(max_df=0.7, min_df=0.01)

In [8]:
X_train_vectorized = count_vectorizer.fit_transform(X_train)
X_test_vectorized = count_vectorizer.transform(X_test)

In [9]:
print(len(count_vectorizer.get_feature_names_out()))
print(count_vectorizer.get_feature_names_out()[:30])
print(count_vectorizer.get_feature_names_out()[-30:])

1401
['ability' 'able' 'absolutely' 'abuse' 'accept' 'access' 'accord'
 'account' 'achieve' 'acquire' 'act' 'action' 'activity' 'actual'
 'actually' 'ad' 'adam' 'add' 'addition' 'additional' 'address' 'admin'
 'administration' 'admit' 'advance' 'advantage' 'advertisement'
 'advertising' 'advice' 'affiliate']
['williams' 'willing' 'win' 'window' 'wink' 'wireless' 'wish' 'woman'
 'wonder' 'wonderful' 'word' 'work' 'worker' 'world' 'worldwide' 'worry'
 'worth' 'wouldn' 'write' 'writer' 'wrong' 'www' 'xml' 'yahoo' 'yeah'
 'year' 'yes' 'yesterday' 'york' 'young']


Проверка на константное предсказание

In [10]:
dummy_clf = DummyClassifier(strategy="most_frequent").fit(X_train_vectorized, y_train)
preds = dummy_clf.predict(X_test_vectorized)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       606
           1       0.00      0.00      0.00       144

    accuracy                           0.81       750
   macro avg       0.40      0.50      0.45       750
weighted avg       0.65      0.81      0.72       750



Логистическая регрессия

In [11]:
pipe = Pipeline(
    steps=[
        ('counter', count_vectorizer),
        ('clf', LogisticRegression())
    ]
).fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       606
           1       0.99      0.90      0.95       144

    accuracy                           0.98       750
   macro avg       0.98      0.95      0.97       750
weighted avg       0.98      0.98      0.98       750



In [12]:
pipe = Pipeline(
    steps=[
        ('tfidf', tfidf_vectorizer),
        ('clf', LogisticRegression())
    ]
).fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       606
           1       0.99      0.81      0.89       144

    accuracy                           0.96       750
   macro avg       0.97      0.91      0.94       750
weighted avg       0.96      0.96      0.96       750



Деревья решений

In [13]:
pipe = Pipeline(
    steps=[
        ('counter', count_vectorizer),
        ('clf', DecisionTreeClassifier())
    ]
).fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96       606
           1       0.81      0.85      0.83       144

    accuracy                           0.93       750
   macro avg       0.89      0.90      0.90       750
weighted avg       0.94      0.93      0.94       750



In [14]:
pipe = Pipeline(
    steps=[
        ('tfidf', tfidf_vectorizer),
        ('clf', DecisionTreeClassifier())
    ]
).fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       606
           1       0.80      0.86      0.83       144

    accuracy                           0.93       750
   macro avg       0.88      0.90      0.89       750
weighted avg       0.93      0.93      0.93       750



Наивный Байесовский классификатор

In [15]:
pipe = Pipeline(
    steps=[
        ('counter', count_vectorizer),
        ('clf', MultinomialNB())
    ]
).fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       606
           1       0.94      0.93      0.94       144

    accuracy                           0.98       750
   macro avg       0.96      0.96      0.96       750
weighted avg       0.98      0.98      0.98       750



In [16]:
pipe = Pipeline(
    steps=[
        ('tfidf', tfidf_vectorizer),
        ('clf', MultinomialNB())
    ]
).fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       606
           1       0.98      0.88      0.93       144

    accuracy                           0.97       750
   macro avg       0.98      0.94      0.96       750
weighted avg       0.97      0.97      0.97       750

